# Network Analysis

Comparing different pretrained models in terms of accuracy and performance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.legend_handler import HandlerTuple
sns.set()

In [ ]:
df = pd.read_csv("networks.csv")
df["Weight"] = df["Weight"].apply(lambda x: x.split(".")[0].replace("_Weights", "").replace("_", " "))
df = df[np.invert(df.duplicated(subset="Weight"))]
df.head()

In [ ]:
best = df.nlargest(10, "Acc@1").nsmallest(5, "GFLOPS")
best["Params"] = best["Params"].astype('int64') 
best

In [ ]:
best.to_latex(index=False, float_format="{:.3f}".format, escape=False)

In [ ]:
noi = df.nlargest(10, "Acc@5").nsmallest(5, "GFLOPS")["Weight"].tolist()
noi.append("ResNet50")
noi.append("AlexNet")
noi.append("Inception V3")
noi

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

first = True

scatter_best = []
scatter_rest = []

col_id = 2

labels = []

for idx, row in df.iterrows():
    if not row["Weight"] in noi:
        if first:
            a = ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', edgecolors ="black", color="None", linewidth=0.3, label="Accuracy top 1", s=1)
            b = ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', edgecolors ="black", color="None", linewidth=0.3, label="Accuracy top 5", s=1)
            labels.append("Accuracy top 1")
            labels.append("Accuracy top 5")
            scatter_rest = [a,b]
            first = False
            
        ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', edgecolors ="black", color="None", linewidth=0.3, s=20)
        ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', edgecolors ="black", color="None", linewidth=0.3, s=20)

for idx, row in df.iterrows():
    if row["Weight"] == noi[0]:
        a = ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', label=row["Weight"], color=f"C{col_id}", edgecolors="black", s=1)
        b = ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', label=row["Weight"], color=f"C{col_id}", edgecolors="black", s=1)
        ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', label=row["Weight"], color=f"C{col_id}", edgecolors="black", s=100)
        ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', label=row["Weight"], color=f"C{col_id}", edgecolors="black", s=100)
        labels.append(row["Weight"])
        scatter_best.append((a, b))
        col_id += 1
    elif row["Weight"] in noi:
        # line = ax.scatter(row['GFLOPS'], row['Acc@1'], marker='.', label=row["Weight"], color=f"C{col_id}", s=1)
        ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', label=row["Weight"], color=f"C{col_id}", s=100)
        ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', label=row["Weight"], color=f"C{col_id}", s=100)
        a = ax.scatter(row['GFLOPS'], row['Acc@1'], marker='^', label=row["Weight"], color=f"C{col_id}", s=1)
        b = ax.scatter(row['GFLOPS'], row['Acc@5'], marker='v', label=row["Weight"], color=f"C{col_id}", s=1)
        labels.append(row["Weight"])
        scatter_best.append((a, b))
        col_id += 1

scatter = scatter_rest + scatter_best
ax.set_xscale("log")
ax.set_title("Accuracy vs. GFLOPS")
ax.legend(handles=scatter, labels=labels, ncols=2, markerscale=10, handler_map={tuple: HandlerTuple(ndivide=None)})
ax.set_xlabel("GFLOPS")
ax.set_ylabel("Accuracy")

plt.savefig("figures/network/networks.png", dpi=300, bbox_inches='tight')
plt.show()